In [1]:
import tensorflow as tf

model_folder = 'models/metrabs_rn18_y4/'
model = tf.saved_model.load(model_folder)

@tf.function()
def my_predict(my_prediction_inputs, **kwargs):
    # input_signature=[                                                  
    #     tf.TensorSpec(shape=(None, None, 3), dtype=tf.uint8),  # image
    #     tf.TensorSpec(shape=(3, 3), dtype=tf.float32),  # intrinsic_matrix
    #     tf.TensorSpec(shape=(5), dtype=tf.float32),  # distortion_coeffs
    #     tf.TensorSpec(shape=(4, 4), dtype=tf.float32),  # extrinsic_matrix
    #     tf.TensorSpec(shape=(3,), dtype=tf.float32),  # world_up_vector
    #     tf.TensorSpec(shape=(), dtype=tf.float32),  # default_fov_degrees
    #     tf.TensorSpec(shape=(), dtype=tf.int32),  # internal_batch_size        
    #     tf.TensorSpec(shape=(), dtype=tf.int32),  # antialias_factor
    #     tf.TensorSpec(shape=(), dtype=tf.int32),  # num_aug
    #     tf.TensorSpec(shape=(), dtype=tf.bool),  # average_aug
    #     tf.TensorSpec(shape=(), dtype=tf.string),  # skeleton
    #     tf.TensorSpec(shape=(), dtype=tf.float32),  # detector_threshold
    #     tf.TensorSpec(shape=(), dtype=tf.float32),  # detector_nms_iou_threshold
    #     tf.TensorSpec(shape=(), dtype=tf.int32),  # max_detections
    #     tf.TensorSpec(shape=(), dtype=tf.bool),  # detector_flip_aug
    #     tf.TensorSpec(shape=(), dtype=tf.bool) # suppress_implausible_poses
    #                ] 

    # inputs = {
    #     'image': my_prediction_inputs,
    #     'intrinsic_matrix': tf.constant(((-1, -1, -1), (-1, -1, -1), (-1, -1, -1)), dtype=tf.float32),
    #     'distortion_coeffs': tf.constant((0, 0, 0, 0, 0), dtype=tf.float32),
    #     'extrinsic_matrix': tf.constant(((1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)), dtype=tf.float32),
    #     'world_up_vector': tf.constant((0, -1, 0), dtype=tf.float32),
    #     'default_fov_degrees': tf.constant(55, dtype=tf.float32), 
    #     'internal_batch_size': tf.constant(64, dtype=tf.int32),
    #     'antialias_factor': tf.constant(1, dtype=tf.int32),
    #     'num_aug': tf.constant(5, dtype=tf.int32),
    #     'average_aug': tf.constant(True, dtype=tf.bool),
    #     'skeleton': tf.constant('', dtype=tf.string),
    #     'detector_threshold': tf.constant(0.3, dtype=tf.float32),
    #     'detector_nms_iou_threshold': tf.constant(0.7, dtype=tf.float32),
    #     'max_detections': tf.constant(-1, dtype=tf.int32),
    #     'detector_flip_aug': tf.constant(False, dtype=tf.bool),
    #     'suppress_implausible_poses': tf.constant(True, dtype=tf.bool)
    # }
    
    prediction = model.detect_poses(my_prediction_inputs)
    return {"prediction": prediction['poses3d']}

my_signatures = my_predict.get_concrete_function(
   my_prediction_inputs=tf.TensorSpec([None,None, 3], dtype=tf.dtypes.uint8, name="image")
)

#signatures = {tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY : 
#                my_signatures}

tf.saved_model.save(model, 'rn18sig', signatures=my_signatures)

INFO:tensorflow:Assets written to: rn18sig/assets


INFO:tensorflow:Assets written to: rn18sig/assets


In [1]:
import os
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(
    max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(
    maximum_cached_engines=100)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir='rn18sig')
    #conversion_params=conversion_params)
converter.convert()
converter.save('tensorrt')

INFO:tensorflow:Linked TensorRT version: (7, 2, 2)
INFO:tensorflow:Loaded TensorRT version: (7, 2, 1)
ERROR:tensorflow:Loaded TensorRT 7.2.1 but linked TensorFlow against TensorRT 7.2.2. TensorRT does not support forward compatibility. It is also required to use the same major version of TensorRT during compilation and runtime.


RuntimeError: Incompatible TensorRT versions